In [7]:
# screener_dma2green_entries.py
"""
DMA(10,20) + strict 2-green screener (ENTRY) with optional indicator filters.
- Scans a list of tickers for the last N days
- Uses tuned params from your grid search (toggle on/off)
- Saves date-wise CSV of signals
- Appends new positions to portfolio/open_positions.csv (avoids duplicates, robust datetime handling)
- Sends a Telegram message summarizing signals (optional)

ENV required for Telegram:
  TELEGRAM_BOT_TOKEN, TELEGRAM_CHAT_ID

Run:
  python screener_dma2green_entries.py

Tip: schedule this daily after market close.
"""

import os
import time
import json
import pathlib
import traceback
import pandas as pd
import numpy as np
import yfinance as yf
import requests
from datetime import datetime, timedelta, timezone
from typing import List, Dict

# -------------------- USER CONFIG --------------------
TICKER_LIST = [
    'BSE.NS','BAJFINANCE.NS','BAJAJFINSV.NS','BDL.NS','BEL.NS','BHARTIARTL.NS',
    'CHOLAFIN.NS','COFORGE.NS','DIVISLAB.NS','DIXON.NS','NYKAA.NS','HDFCBANK.NS',
    'HDFCLIFE.NS','ICICIBANK.NS','INDHOTEL.NS','INDIGO.NS','KOTAKBANK.NS','MFSL.NS',
    'MAXHEALTH.NS','MAZDOCK.NS','MUTHOOTFIN.NS','PAYTM.NS','PERSISTENT.NS','SBICARD.NS',
    'SBILIFE.NS','SRF.NS','SHREECEM.NS','SOLARINDS.NS','TVSMOTOR.NS','UNITDSPR.NS'
]

START_DATE = "2015-01-01"
END_DATE = None  # None = today

# How many recent bars to scan for entries (date-wise output)
SCAN_LOOKBACK_DAYS = 50

# Tuned params (toggle on/off, same shape as backtest)
PARAMS = {
    "combination_mode": "all",  # 'all' or 'any'

    "rsi_enabled": True,
    "rsi_min": 45,

    "adx_enabled": False,
    "adx_length": 14,
    "adx_min": 20,

    "sma_enabled": False,
    "sma_length": 50,

    "bb_enabled": False,
    "bb_length": 20,
    "bb_stddev": 2.0,

    "macd_enabled": False,
    "macd_fast": 12,
    "macd_slow": 26,
    "macd_signal": 9,

    # Exit/management settings to record with the position (for the exit script)
    "protective_exit": False,
    "use_hard_stop": True,
    "hard_stop_pct": 5.0,
    "use_trailing_stop": False,
    "trailing_stop_pct": 10.0,
    "max_hold_days": 10
}

DMA_FAST = 10
DMA_SLOW = 20

# Telegram
ENABLE_TELEGRAM = True
TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN", "")
TELEGRAM_CHAT_ID = os.getenv("TELEGRAM_CHAT_ID", "")

# Paths
CACHE_DIR = pathlib.Path("cache")
SIGNALS_DIR = pathlib.Path("signals")
PORTFOLIO_DIR = pathlib.Path("portfolio")
OPEN_POSITIONS_CSV = PORTFOLIO_DIR / "open_positions.csv"

for p in [CACHE_DIR, SIGNALS_DIR, PORTFOLIO_DIR]:
    p.mkdir(parents=True, exist_ok=True)

# -------------------- HELPERS --------------------
def normalize_df_columns(df: pd.DataFrame) -> pd.DataFrame:
    if hasattr(df, "columns") and getattr(df.columns, "nlevels", 1) > 1:
        df.columns = ["_".join([str(c) for c in col if c is not None]).strip() for c in df.columns.values]
    cols = list(df.columns)
    mapping = {}
    lower_map = {c.lower(): c for c in cols}
    for name in ['close', 'high', 'low', 'open', 'volume']:
        if name in lower_map:
            mapping[lower_map[name]] = name.capitalize()
        else:
            match = next((c for c in cols if name in c.lower()), None)
            if match:
                mapping[match] = name.capitalize()
    if mapping:
        df = df.rename(columns=mapping)
    return df

def collapse_duplicate_columns_take_first(df: pd.DataFrame) -> pd.DataFrame:
    if df.columns.duplicated().any():
        df = df.groupby(df.columns, axis=1).first()
    return df

def _cache_path_for_ticker(ticker: str) -> pathlib.Path:
    safe = ticker.replace('/', '_').replace(':', '_')
    return CACHE_DIR / f"{safe}.csv"

def _read_cache(ticker: str) -> pd.DataFrame:
    p = _cache_path_for_ticker(ticker)
    if p.exists() and p.stat().st_size > 0:
        try:
            return pd.read_csv(p, index_col=0, parse_dates=True)
        except Exception:
            try:
                p.unlink()
            except Exception:
                pass
    return None

def _write_cache(ticker: str, df: pd.DataFrame) -> None:
    p = _cache_path_for_ticker(ticker)
    try:
        df.to_csv(p)
    except Exception:
        pass

def download_with_retry(ticker: str, start: str, end: str, interval="1d", max_retries=3, backoff_sec=2):
    attempt = 0
    while attempt < max_retries:
        try:
            df = yf.download(ticker, start=start, interval=interval, end=end,
                             auto_adjust=True, progress=False, threads=True, multi_level_index=False)
            return df
        except Exception as e:
            attempt += 1
            time.sleep(backoff_sec * attempt)
    return None

def sma(series: pd.Series, length: int) -> pd.Series:
    return series.rolling(length).mean()

def rsi(df: pd.DataFrame, length=14, column='Close') -> pd.DataFrame:
    s = df[column]
    delta = s.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/length, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/length, adjust=False).mean()
    rs = avg_gain / (avg_loss.replace(0, np.nan))
    df['RSI'] = (100 - (100 / (1 + rs))).fillna(50)
    return df

def macd(df: pd.DataFrame, fast=12, slow=26, signal=9, column='Close') -> pd.DataFrame:
    ema_fast = df[column].ewm(span=fast, adjust=False).mean()
    ema_slow = df[column].ewm(span=slow, adjust=False).mean()
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    df['MACD'] = macd_line
    df['MACD_signal'] = signal_line
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']
    return df

def adx(df: pd.DataFrame, n=14) -> pd.DataFrame:
    high = df['High']; low = df['Low']; close = df['Close']
    tr1 = high - low
    tr2 = (high - close.shift(1)).abs()
    tr3 = (low - close.shift(1)).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    up_move = high.diff(); down_move = -low.diff()
    plus_dm = np.where((up_move > down_move) & (up_move > 0), up_move, 0.0)
    minus_dm = np.where((down_move > up_move) & (down_move > 0), down_move, 0.0)
    tr_smooth = pd.Series(tr).rolling(window=n).sum()
    plus_dm_smooth = pd.Series(plus_dm).rolling(window=n).sum()
    minus_dm_smooth = pd.Series(minus_dm).rolling(window=n).sum()
    plus_di = 100 * (plus_dm_smooth / tr_smooth).replace([np.inf, -np.inf], 0).fillna(0)
    minus_di = 100 * (minus_dm_smooth / tr_smooth).replace([np.inf, -np.inf], 0).fillna(0)
    dx = (abs(plus_di - minus_di) / (plus_di + minus_di)).replace([np.inf, -np.inf], 0) * 100
    df['+DI'] = plus_di; df['-DI'] = minus_di; df['ADX'] = dx.rolling(window=n).mean()
    return df

def bbands(df: pd.DataFrame, length=20, stddev=2.0, column='Close') -> pd.DataFrame:
    mid = df[column].rolling(length).mean()
    std = df[column].rolling(length).std()
    df['BB_middle'] = mid
    df['BB_upper'] = mid + stddev*std
    df['BB_lower'] = mid - stddev*std
    return df

def two_green_strict(df: pd.DataFrame) -> pd.Series:
    o, h, l, c = df['Open'], df['High'], df['Low'], df['Close']
    green_prev = df['Close'].shift(1) > df['Open'].shift(1)
    green_now  = df['Close'] > df['Open']
    cond_strict = (o > o.shift(1)) & (h > h.shift(1)) & (l > l.shift(1)) & (c > c.shift(1))
    return (green_prev & green_now & cond_strict)

def build_base_signals(df: pd.DataFrame, dma_fast=10, dma_slow=20) -> pd.DataFrame:
    df['DMA_fast'] = sma(df['Close'], dma_fast)
    df['DMA_slow'] = sma(df['Close'], dma_slow)
    df['dma_cross_up'] = (df['DMA_fast'] > df['DMA_slow']) & (df['DMA_fast'].shift(1) <= df['DMA_slow'].shift(1))
    df['two_green_strict'] = two_green_strict(df)
    return df

def indicator_entry_filter(row: pd.Series, params: Dict) -> Dict[str, bool]:
    checks = {}
    if params.get('rsi_enabled', False):
        checks['rsi'] = (row.get('RSI', np.nan) >= params.get('rsi_min', 45))
    if params.get('adx_enabled', False):
        adx_val = row.get('ADX', 0); plus = row.get('+DI', 0); minus = row.get('-DI', 0)
        checks['adx'] = (adx_val >= params.get('adx_min', 20)) and (plus > minus)
    if params.get('sma_enabled', False):
        smalen = params.get('sma_length', 50)
        checks['sma'] = (row['Close'] >= row.get(f'SMA_{smalen}', np.nan))
    if params.get('bb_enabled', False):
        checks['bb'] = (row['Close'] >= row.get('BB_middle', np.nan))
    if params.get('macd_enabled', False):
        checks['macd'] = (row.get('MACD', 0) > row.get('MACD_signal', 0)) and (row.get('MACD_hist', 0) > 0)
    return checks

def combine_indicator_signals(ind_results: Dict[str, bool], mode='all') -> bool:
    if not ind_results:
        return True
    vals = list(ind_results.values())
    return all(vals) if mode == 'all' else any(vals)

def get_stock_data(ticker: str, start: str, end: str, params: Dict) -> pd.DataFrame:
    raw = _read_cache(ticker)
    if raw is None:
        raw = download_with_retry(ticker, start, end, interval="1d", max_retries=3, backoff_sec=2)
        if raw is None or raw.empty:
            return None
        raw = normalize_df_columns(raw)
        raw = collapse_duplicate_columns_take_first(raw)
        _write_cache(ticker, raw)

    df = raw.copy()
    df = normalize_df_columns(df)
    df = collapse_duplicate_columns_take_first(df)

    for c in ['Close','High','Low','Open','Volume']:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce')

    df = build_base_signals(df, dma_fast=DMA_FAST, dma_slow=DMA_SLOW)

    if params.get('rsi_enabled', False):
        df = rsi(df, length=14)
    if params.get('macd_enabled', False):
        df = macd(df, fast=params.get('macd_fast',12), slow=params.get('macd_slow',26), signal=params.get('macd_signal',9))
    if params.get('adx_enabled', False):
        df = adx(df, n=params.get('adx_length',14))
    if params.get('sma_enabled', False):
        smalen = params.get('sma_length',50)
        df[f"SMA_{smalen}"] = df['Close'].rolling(smalen).mean()
    if params.get('bb_enabled', False):
        df = bbands(df, length=params.get('bb_length',20), stddev=params.get('bb_stddev',2.0))

    df.dropna(inplace=True)
    return df if not df.empty else None

def find_entries(df: pd.DataFrame, params: Dict, lookback_days: int) -> pd.DataFrame:
    """Return rows where entry condition is met within the last N bars. Stores date as ISO string."""
    if df is None or df.empty:
        return pd.DataFrame()

    df_tail = df.tail(lookback_days)
    rows = []
    for idx, row in df_tail.iterrows():
        base_entry = bool(row['dma_cross_up'] and row['two_green_strict'])
        ind_results = indicator_entry_filter(row, params)
        enabled_any = any([params.get(k) for k in ['rsi_enabled','macd_enabled','adx_enabled','sma_enabled','bb_enabled']])
        ind_ok = combine_indicator_signals(ind_results, mode=params.get('combination_mode','all'))
        if base_entry and (ind_ok if enabled_any else True):
            date_iso = pd.to_datetime(idx).strftime('%Y-%m-%d')
            rows.append({
                "date": date_iso,
                "close": float(row['Close']),
                "dma10": float(row['DMA_fast']),
                "dma20": float(row['DMA_slow']),
                "rsi": float(row.get('RSI', np.nan)) if 'RSI' in df.columns else np.nan,
                "adx": float(row.get('ADX', np.nan)) if 'ADX' in df.columns else np.nan
            })
    return pd.DataFrame(rows)

def send_telegram(text: str) -> None:
    if not ENABLE_TELEGRAM:
        print("[DRY] Telegram disabled. Message would be:\n", text)
        return
    token = TELEGRAM_BOT_TOKEN
    chat_id = TELEGRAM_CHAT_ID
    if not token or not chat_id:
        print("Telegram ENV missing; skipping send.")
        print(text)
        return
    url = f"https://api.telegram.org/bot{token}/sendMessage"
    try:
        requests.post(url, data={"chat_id": chat_id, "text": text, "parse_mode": "Markdown"})
    except Exception as e:
        print("Telegram send error:", e)
        print(text)

def _init_open_positions_df() -> pd.DataFrame:
    cols = [
        "ticker","entry_date","entry_price",
        "protective_exit","use_hard_stop","hard_stop_pct",
        "use_trailing_stop","trailing_stop_pct","max_hold_days",
        "peak_price"
    ]
    return pd.DataFrame(columns=cols)

def append_open_positions(ticker: str, signal_rows: pd.DataFrame, params: Dict) -> int:
    """Append new positions for this ticker for each signal date that's not already open."""
    if signal_rows.empty:
        return 0

    # Load current open positions
    if OPEN_POSITIONS_CSV.exists():
        open_df = pd.read_csv(OPEN_POSITIONS_CSV)
    else:
        open_df = _init_open_positions_df()

    # Normalize types
    if 'entry_date' in open_df.columns:
        open_df['entry_date'] = pd.to_datetime(open_df['entry_date'], errors='coerce').dt.date
    if 'ticker' in open_df.columns:
        open_df['ticker'] = open_df['ticker'].astype(str)

    existing_keys = set(zip(open_df.get('ticker', pd.Series(dtype=str)),
                            open_df.get('entry_date', pd.Series(dtype='datetime64[ns]'))))

    new_rows = []
    for _, r in signal_rows.iterrows():
        # r['date'] is ISO str; keep it as ISO to avoid dtype surprises in CSV
        date_iso = str(r['date'])
        key = (ticker, pd.to_datetime(date_iso).date())
        if key in existing_keys:
            continue
        new_rows.append({
            "ticker": ticker,
            "entry_date": date_iso,  # keep ISO string; we'll parse on read
            "entry_price": float(r['close']),
            "protective_exit": bool(params.get('protective_exit', True)),
            "use_hard_stop": bool(params.get('use_hard_stop', False)),
            "hard_stop_pct": float(params.get('hard_stop_pct', 5.0)),
            "use_trailing_stop": bool(params.get('use_trailing_stop', False)),
            "trailing_stop_pct": float(params.get('trailing_stop_pct', 10.0)),
            "max_hold_days": int(params.get('max_hold_days', 10)),
            "peak_price": float(r['close'])
        })

    if not new_rows:
        return 0

    # Align columns to avoid FutureWarning behavior changes
    add_df = pd.DataFrame(new_rows)
    for col in open_df.columns:
        if col not in add_df.columns:
            add_df[col] = np.nan
    add_df = add_df[open_df.columns] if list(open_df.columns) else add_df

    open_df = pd.concat([open_df, add_df], ignore_index=True)
    open_df.to_csv(OPEN_POSITIONS_CSV, index=False)
    return len(new_rows)

def main():
    end = END_DATE
    if end is None:
        end = (datetime.now(timezone.utc) + timedelta(days=1)).strftime("%Y-%m-%d")

    date_buckets = {}  # date_iso -> [tickers]

    total_new_positions = 0
    for t in TICKER_LIST:
        try:
            df = get_stock_data(t, START_DATE, end, PARAMS)
            if df is None:
                continue
            signals = find_entries(df, PARAMS, SCAN_LOOKBACK_DAYS)
            if signals.empty:
                continue

            # group for telegram
            for _, r in signals.iterrows():
                d = r['date']
                date_buckets.setdefault(d, []).append(t)

            # append to open positions
            added = append_open_positions(t, signals, PARAMS)
            total_new_positions += added

        except Exception as e:
            print(f"[{t}] error: {e}\n{traceback.format_exc()}")

    # Save date-wise CSV
    for d, names in sorted(date_buckets.items()):
        outpath = SIGNALS_DIR / f"entries_{d}.csv"
        pd.DataFrame({"ticker": sorted(names)}).to_csv(outpath, index=False)

    # Telegram summary
    if date_buckets:
        lines = ["*DMA(10,20)+2-Green Entries*"]
        for d, names in sorted(date_buckets.items()):
            lines.append(f"*{d}*  — {', '.join(sorted(names))}")
        lines.append(f"\nAdded to open positions: {total_new_positions}")
        send_telegram("\n".join(lines))
        print("Telegram sent.")
    else:
        print("No entries in lookback window.")

    print(f"Open positions file: {OPEN_POSITIONS_CSV.resolve()}")

if __name__ == "__main__":
    main()


Telegram ENV missing; skipping send.
*DMA(10,20)+2-Green Entries*
*2025-08-13*  — INDIGO.NS
*2025-08-14*  — HDFCLIFE.NS, MFSL.NS, SOLARINDS.NS
*2025-09-10*  — SRF.NS
*2025-09-15*  — CHOLAFIN.NS
*2025-09-16*  — BDL.NS
*2025-09-17*  — BHARTIARTL.NS, KOTAKBANK.NS
*2025-09-18*  — COFORGE.NS, HDFCBANK.NS
*2025-09-19*  — SBILIFE.NS

Added to open positions: 4
Telegram sent.
Open positions file: /Users/hemank/Documents/github/trading/swing/00 final/00 DMA/portfolio/open_positions.csv
